# Language Level Topics

## Strict computation

### `seq`

In [67]:
:i `seq`

seq :: a -> b -> b 	-- Defined in ‘GHC.Prim’
infixr 0 `seq`

In [68]:
seq 1 2

2

In [69]:
[1,2] `seq` [3,4]

Line 1: Redundant seq
Found:
[1, 2] `seq` [3, 4]
Why not:
[3, 4]

[3,4]

So `seq` is similar to the Functor's `const` function (see following example):
```haskell
    ⊥ `seq` b = ⊥
    a `seq` b = b
```
where `⊥` is the [bottom](https://wiki.haskell.org/Bottom) .

In [70]:
[] `const` [3,4]

Line 1: Evaluate
Found:
[] `const` [3, 4]
Why not:
[]

[]

In [71]:
:t undefined

undefined :: forall {a}. a

In [72]:
[] `seq` [3,4]

Line 1: Redundant seq
Found:
[] `seq` [3, 4]
Why not:
[3, 4]

[3,4]

In [73]:
import Data.Void

:i Void

type Void :: *
data Void
  	-- Defined in ‘Data.Void’
instance [safe] Show Void -- Defined in ‘Data.Void’
instance [safe] Eq Void -- Defined in ‘Data.Void’
instance [safe] Ord Void -- Defined in ‘Data.Void’
instance [safe] Semigroup Void -- Defined in ‘Data.Void’
instance [safe] Read Void -- Defined in ‘Data.Void’

In [74]:
() `seq` [3,4]

Line 1: Redundant seq
Found:
() `seq` [3, 4]
Why not:
[3, 4]

[3,4]

So `[]` is not the bottom for `list`.

In [75]:
[1..] `seq` [3,4]

[3,4]

This shows non-terminating is not the bottom for `list`. `undefined` does do the job as it returns `undefined` for "undefine `seq` a", in the same token `Void` is the bottom as well, except the it reports error when executing `undefined` or `Void`.

### `$!`
`$!` is strict operation

In [76]:
:t ($!)

($!) :: forall {a} {b}. (a -> b) -> a -> b

In [77]:
:i ($!)

($!) :: (a -> b) -> a -> b 	-- Defined in ‘GHC.Base’
infixr 0 $!

In [78]:
id $! [1,2]

Line 1: Redundant $!
Found:
id $! [1, 2]
Why not:
[1, 2]Line 1: Redundant $!
Found:
id $! [1, 2]
Why not:
id [1, 2]

[1,2]

In [79]:
(++[3,4]) $! [1,2]

Line 1: Redundant $!
Found:
(++ [3, 4]) $! [1, 2]
Why not:
(++ [3, 4]) [1, 2]

[1,2,3,4]

In [80]:
--const undefined [1,2]
--undefined `seq` [1,2]

In [81]:
--const Void [1,2]
--Void `seq` [1,2]

`f $! x = f x`

Interesting information on `seq`: [Why seq is bad](https://stackoverflow.com/questions/12687392/why-is-seq-bad).

### Strict computation
Strict computation for `f` can be forced by using `f $!`.
```haskell
f $! x  = x `seq` f x
```

In [82]:
id $! [1,2]

Line 1: Redundant $!
Found:
id $! [1, 2]
Why not:
[1, 2]Line 1: Redundant $!
Found:
id $! [1, 2]
Why not:
id [1, 2]

[1,2]

One application of this technique is the definiton of `foldr` based on `foldl`:
```haskell
foldr' :: (a -> b -> b) -> b -> [a] -> b
foldr' f z0 xs = foldl f' id xs z0
  where f' k x z = k $! f x z
```